In [ ]:
import pandas as pd
import numpy as np

dataset link: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

Importing dataset

In [ ]:
raw = pd.read_csv("SMSSpamCollection", sep='\t',header=None, names=['isSpam', 'message'])

In [ ]:
raw.head(10)

,isSpam,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [ ]:
len(raw)

5572

remove duplicates

In [ ]:

raw.drop_duplicates(subset=['message'], inplace=True)
len(raw)


5169

Data cleaning

In [ ]:
import string
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

remove punctuation and stopwords

In [ ]:
raw['processed'] = raw['message'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stopwords and word not in punctuation]))

In [ ]:

raw['processed'] = raw['processed'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

#or
# def pre_process(sms):
#     remove_punct = "".join([word.lower() for word in sms if word not in punctuation])
#     tokenize = nltk.tokenize.word_tokenize(remove_punct)
#     remove_stopwords = [word for word in tokenize if word not in stopwords]
#     return remove_stopwords
# create a separate column
# data['processed'] = data['message'].apply(lambda x: pre_process(x))

In [ ]:
raw.head()

,isSpam,message,processed
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'label' is the column containing 'spam' and 'ham'
raw['isSpam'] = LabelEncoder().fit_transform(raw['isSpam'])

In [ ]:
raw.head()

,isSpam,message,processed
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
x = raw['message'].values
y = raw['isSpam'].values

In [ ]:
max_words = 1000
max_len = 150

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
X_seq = tokenizer.texts_to_sequences(x)
X_padded = pad_sequences(X_seq, maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

NEURAL NETWORK

In [ ]:
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=100))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 4
batch_size = 64
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/4
65/65 [==============================] - 8s 123ms/step - loss: 0.0966 - accuracy: 0.9678 - val_loss: 0.0845 - val_accuracy: 0.9758
Epoch 2/4
65/65 [==============================] - 9s 133ms/step - loss: 0.0535 - accuracy: 0.9857 - val_loss: 0.0793 - val_accuracy: 0.9768
Epoch 3/4
65/65 [==============================] - 9s 137ms/step - loss: 0.0416 - accuracy: 0.9896 - val_loss: 0.0762 - val_accuracy: 0.9787
Epoch 4/4
65/65 [==============================] - 9s 132ms/step - loss: 0.0334 - accuracy: 0.9927 - val_loss: 0.0753 - val_accuracy: 0.9787


EVALUATE

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

33/33 [==============================] - 1s 19ms/step - loss: 0.0753 - accuracy: 0.9787
Test Accuracy: 97.87%


TEST

In [110]:
new_messages = ["Free entry to win a prize!", "Hey, how are you?"]
new_messages_seq = tokenizer.texts_to_sequences(new_messages)
new_messages_padded = pad_sequences(new_messages_seq, maxlen=max_len)
predictions = model.predict(new_messages_padded)
predicted_labels = (predictions > 0.5).astype(int).flatten()

for message, label in zip(new_messages, predicted_labels):
    print(f"Message: {message}, Predicted Label: {'Spam' if label == 1 else 'Not Spam'}")

1/1 [==============================] - 0s 22ms/step
Message: Free entry to win a prize!, Predicted Label: Spam
Message: Hey, how are you?, Predicted Label: Not Spam


So, the accuracy of the model is 97.87% and precision is 100%.

**saving the model**

In [ ]:
model.save('spam_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#code to load the model again in some file
from tensorflow.keras.models import load_model

# Load the model from the file
loaded_model = load_model('spam_detection_model.h5')

In [ ]:
#saving as an API
import tensorflow as tf
model.save('spam_detection_saved_model', save_format='tf')

# Load the model from the SavedModel format
loaded_model = tf.keras.models.load_model('spam_detection_saved_model')

**CONCLUSION**


An improvised dataset with more spam types.